## **Dataset Overview**

- **Data Range:** From 2022-01-01 to 2023-09-01
- **Campaign Markets:** UK(focus) / DE(reference)
- **Observation:** a growing share of conversions occurring outside the UK
- **Analysis tool:** Datacamp > Datalab > SQL


## Analysis framework
In order to find out the potential factors for non-UK conversion increase, this analysis will dive into **_Geo-Targeting Accuracy_** from the location targeting perspective and **_Cross-Border Traffic_** from target audience perspective.

How to define Geo-Targetng / Cross-Border Traffic?
- **conversion country vs. asset country mismatch:** Cross-border traffic occurs when the **_conversion_country (where the conversion happens)_** is different from the **_asset_country (where the campaign is targeted)_**.
- **country code by phone mismatch:** A strong indicator of cross-border traffic is when the **_country_code_by_phone (user’s phone number)_** differs from the **_conversion_country_**. This implies that a user from another country completed a conversion in a different country.



Therefore, in this case, there could be 2 conditions:


1. Condition 1 - Geo-targeting issue: **_accesst_country != conversion_coutry_** > publisher
     - accesst_country = UK
     - conversion_country != UK


2. Condition 2 - target audience : **_country_code_by_phone != conversion_coutry_** > cross-border traffic
- Scenario 1: checks for UK users converting outside the UK
    - accesst_country = UK
    - country_code_by_phone = UK
    - conversion_country != UK
- Scenario 2: checks for non-UK users converting outside the UK, even though the campaign is targeted at the UK
    - accesst_country = UK
    - country_code_by_phone = !UK
    - conversion_country != UK

## Data Analysis - Trend Analysis
### **1. Showcase the UK/Non-UK conversion trend from targeted UK campaigns.**
   - Why do I want to know?
      - The campaign targets UK users, but increasing non-UK conversions could signal geo-targeting issues, platform behavior anomalies, or cross-border traffic. Month-over-month (MoM) analysis helps identify when this trend began and its severity, revealing if it's an anomaly or long-standing issue.
   - So what?
        - **Budget Inefficiency:** Marketing spend intended for UK users is being wasted on non-UK conversions, reducing overall ROI.
        - **Geo-Targeting Accuracy:** Misconfigured geo-targeting impacts UK user reach and conversions
        - **Cross-Border Traffic:** If non-UK conversions are genuine interest, it may suggest potential for expansion, but requires strategic adjustments


### **Findings**
- **UK conversion decline:** UK conversions dropped from 83.5% in January 2022 to just 28.2% by September 2023, indicating a sharp decline in UK-based conversions over time.
- **Non-UK conversion growth:** Non-UK conversions increased steadily, growing from 16.5% in January 2022 to 71.8% by September 2023, highlighting a significant shift in audience location.

As non-UK conversions increased, UK conversions decreased in proportion, suggesting potential cannibalization. 

In [14]:
-- UK vs. Non-UK conversion Trend Overview
SELECT 
    month,
    SUM(CASE WHEN conversion_country = 'UK' THEN conversions ELSE 0 END) AS UK_Conversions,
    SUM(CASE WHEN conversion_country = 'UK' THEN conversions ELSE 0 END) / SUM(conversions) AS UK_Conversion_Share,
    SUM(CASE WHEN conversion_country != 'UK' THEN conversions ELSE 0 END) AS Non_UK_Conversions,
    SUM(CASE WHEN conversion_country != 'UK' THEN conversions ELSE 0 END) / SUM(conversions) AS Non_UK_Conversion_Share
FROM 'campaigns.csv'
WHERE asset_country = 'UK'
GROUP BY month
ORDER BY month ASC;

,month,UK_Conversions,UK_Conversion_Share,Non_UK_Conversions,Non_UK_Conversion_Share
0,2022-01-01 00:00:00+00:00,13229.0,0.835217,2610.0,0.164783
1,2022-02-01 00:00:00+00:00,11274.0,0.783025,3124.0,0.216975
2,2022-03-01 00:00:00+00:00,11840.0,0.751603,3913.0,0.248397
3,2022-04-01 00:00:00+00:00,9896.0,0.702591,4189.0,0.297409
4,2022-05-01 00:00:00+00:00,9854.0,0.668113,4895.0,0.331887
5,2022-06-01 00:00:00+00:00,9879.0,0.594333,6743.0,0.405667
6,2022-07-01 00:00:00+00:00,10328.0,0.595720,7009.0,0.404280
7,2022-08-01 00:00:00+00:00,7947.0,0.597744,5348.0,0.402256
8,2022-09-01 00:00:00+00:00,8836.0,0.599742,5897.0,0.400258
9,2022-10-01 00:00:00+00:00,3480.0,0.284663,8745.0,0.715337


In [15]:
-- explore the data in the MoM table
WITH campaign_data AS (
    SELECT 
        month,
        SUM(CASE WHEN conversion_country = 'UK' THEN conversions ELSE 0 END) AS UK_Conversions,
        ROUND(SUM(CASE WHEN conversion_country = 'UK' THEN conversions ELSE 0 END) /
                      SUM(conversions), 4) AS UK_Conversion_Share,
        SUM(CASE WHEN conversion_country != 'UK' THEN conversions ELSE 0 END) AS Non_UK_Conversions,
        ROUND(SUM(CASE WHEN conversion_country != 'UK' THEN conversions ELSE 0 END) /
                      SUM(conversions), 4) AS Non_UK_Conversion_Share
    FROM 'campaigns.csv'
    WHERE asset_country = 'UK'
    GROUP BY month
)

SELECT 
    month,
    UK_Conversions,
	-- MoM for UK Conversions
    UK_Conversions - LAG(UK_Conversions, 1) OVER (ORDER BY month) AS UK_Conversions_MoM,
    CONCAT(ROUND(UK_Conversion_Share * 100, 2), '%') AS UK_Conversion_Share,
    -- MoM for UK Conversion Share
    CONCAT(ROUND((UK_Conversion_Share - LAG(UK_Conversion_Share, 1) OVER (ORDER BY month)) * 100, 2), '%') AS UK_Conversion_Share_MoM,
    Non_UK_Conversions,
    -- MoM for Non-UK Conversions
    Non_UK_Conversions - LAG(Non_UK_Conversions, 1) OVER (ORDER BY month) AS Non_UK_Conversions_MoM,
    CONCAT(ROUND(Non_UK_Conversion_Share * 100, 2), '%') AS Non_UK_Conversion_Share,
    -- MoM for Non-UK Conversion Share
    CONCAT(ROUND((Non_UK_Conversion_Share - LAG(Non_UK_Conversion_Share, 1) OVER (ORDER BY month)) * 100, 2), '%') AS Non_UK_Conversion_Share_MoM
FROM campaign_data
ORDER BY 1;

,month,UK_Conversions,UK_Conversions_MoM,UK_Conversion_Share,UK_Conversion_Share_MoM,Non_UK_Conversions,Non_UK_Conversions_MoM,Non_UK_Conversion_Share,Non_UK_Conversion_Share_MoM
0,2022-01-01 00:00:00+00:00,13229.0,NaN,83.52%,%,2610.0,NaN,16.48%,%
1,2022-02-01 00:00:00+00:00,11274.0,-1955.0,78.3%,-5.22%,3124.0,514.0,21.7%,5.22%
2,2022-03-01 00:00:00+00:00,11840.0,566.0,75.16%,-3.14%,3913.0,789.0,24.84%,3.14%
3,2022-04-01 00:00:00+00:00,9896.0,-1944.0,70.26%,-4.9%,4189.0,276.0,29.74%,4.9%
4,2022-05-01 00:00:00+00:00,9854.0,-42.0,66.81%,-3.45%,4895.0,706.0,33.19%,3.45%
5,2022-06-01 00:00:00+00:00,9879.0,25.0,59.43%,-7.38%,6743.0,1848.0,40.57%,7.38%
6,2022-07-01 00:00:00+00:00,10328.0,449.0,59.57%,0.14%,7009.0,266.0,40.43%,-0.14%
7,2022-08-01 00:00:00+00:00,7947.0,-2381.0,59.77%,0.2%,5348.0,-1661.0,40.23%,-0.2%
8,2022-09-01 00:00:00+00:00,8836.0,889.0,59.97%,0.2%,5897.0,549.0,40.03%,-0.2%
9,2022-10-01 00:00:00+00:00,3480.0,-5356.0,28.47%,-31.5%,8745.0,2848.0,71.53%,31.5%


## Data Analysis: Geo-targeting accuracy
By comparing the publishers vs. with the **_conversion_country_** (where the conversion occurred) and **_asset_country_** (marketing targeted country - in this case it's uk), It allows us to detect if there is any potential geo-targeting inefficiencies.

Therefore, I'll dive into condition 1.
1. Condition 1: **_accesst_country != conversion_coutry_** > Geo-targeting issue > publisher
     - accesst_country = UK
     - conversion_country != UK


### **2.  Evaluate the performance of geo-targeting from the publishers for UK and DE.**
- Why Do I Want to Know?
    - I want to see if specific publishers are causing conversions outside the target market, which could indicate geo-targeting issues.

- So What?
    - If certain publishers are off-target, it means wasted ad spend. Identifying them lets us fix the issue or reallocate budget to improve campaign performance.



### **Findings**
- **Higher UK Mismatches:** UK has consistently higher mismatched conversion rates than DE. For example, PublisherS has a 54.6% UK mismatch vs. 21.8% in DE, and PublisherT shows 52% in the UK vs. 28% in DE.
- **DE Performs Better:** DE generally shows lower mismatched rates. PublisherK has 47% UK mismatches but only 26% in DE

### **Recommendations:**
- Focus on urgent publishers with the highest mismatch rates to refine targeting strategies.

In [1]:
-- How mismatches look like from publisher
SELECT 
    publisher,
    SUM(CASE WHEN asset_country = 'UK' AND conversion_country = 'UK' THEN conversions ELSE 0 END) AS UK_Conversions,
    CONCAT(ROUND(SUM(CASE WHEN asset_country = 'UK' AND conversion_country = 'UK' THEN conversions ELSE 0 END) / SUM(CASE WHEN asset_country = 'UK' THEN conversions ELSE 0 END) * 100, 2), '%') AS UK_Conversion_Share,
    SUM(CASE WHEN asset_country = 'UK' AND conversion_country != 'UK' THEN conversions ELSE 0 END) AS Non_UK_Conversions,
    CONCAT(ROUND(SUM(CASE WHEN asset_country = 'UK' AND conversion_country != 'UK' THEN conversions ELSE 0 END) / SUM(CASE WHEN asset_country = 'UK' THEN conversions ELSE 0 END) * 100, 2), '%') AS Non_UK_Conversion_Share,
    SUM(CASE WHEN asset_country = 'DE' AND conversion_country = 'DE' THEN conversions ELSE 0 END) AS DE_Conversions,
    CONCAT(ROUND(SUM(CASE WHEN asset_country = 'DE' AND conversion_country = 'DE' THEN conversions ELSE 0 END) / SUM(CASE WHEN asset_country = 'DE' THEN conversions ELSE 0 END) * 100, 2), '%') AS DE_Conversion_Share,
    SUM(CASE WHEN asset_country = 'DE' AND conversion_country != 'DE' THEN conversions ELSE 0 END) AS Non_DE_Conversions,
    CONCAT(ROUND(SUM(CASE WHEN asset_country = 'DE' AND conversion_country != 'DE' THEN conversions ELSE 0 END) / SUM(CASE WHEN asset_country = 'DE' THEN conversions ELSE 0 END) * 100, 2), '%') AS Non_DE_Conversion_Share
FROM 'campaigns.csv'
GROUP BY publisher
ORDER BY UK_Conversions DESC;


,publisher,UK_Conversions,UK_Conversion_Share,Non_UK_Conversions,Non_UK_Conversion_Share,DE_Conversions,DE_Conversion_Share,Non_DE_Conversions,Non_DE_Conversion_Share
0,PublisherB,28937.0,51.25%,27529.0,48.75%,46432.0,77.72%,13310.0,22.28%
1,PublisherA,19774.0,66.91%,9779.0,33.09%,43020.0,91.82%,3831.0,8.18%
2,PublisherS,19123.0,45.39%,23007.0,54.61%,15430.0,78.14%,4317.0,21.86%
3,PublisherK,15155.0,52.03%,13972.0,47.97%,16236.0,73.92%,5727.0,26.08%
4,PublisherN,9517.0,93.6%,651.0,6.4%,129.0,82.69%,27.0,17.31%
5,PublisherI,8270.0,77.7%,2373.0,22.3%,2931.0,87.83%,406.0,12.17%
6,PublisherD,7315.0,51.8%,6807.0,48.2%,0.0,%,0.0,%
7,PublisherT,6034.0,47.67%,6623.0,52.33%,7809.0,71.77%,3071.0,28.23%
8,PublisherG,5212.0,45.98%,6123.0,54.02%,5295.0,76.69%,1609.0,23.31%
9,PublisherO,4849.0,55.0%,3967.0,45.0%,2494.0,74.94%,834.0,25.06%


### **3.  Evaluate the conversion performance breakdown by the attribution type for UK and DE.**
- Why Do I Want to Know?
    - I want to see conversion performance in each attribution type, which could indicate geo-targeting issues.

- So What?
    - If certain attribution_type are off-target, it means wasted ad spend. Identifying them lets us fix the issue or reallocate budget to improve campaign performance.

### **Findings:**
- **UK Conversions:**
    - Install: 51.79% of conversions come from UK users, while 48.21% are non-UK, indicating that we need to require a better review for better geo-targeting.
    - Retargeting: Strong UK focus with 72.42% of conversions.
    - Reattribution: 88.40% of UK conversions highlight successful win-back efforts.



- **DE Conversions:**
    -  Install: 80.56% of conversions are local, showing highly effective acquisition campaigns.
    -  Retargeting: 90.54% of conversions come from within Germany, indicating strong re-engagement.



In [2]:
-- Check attribution_type for an overview
SELECT 
    attribution_type,
    SUM(CASE WHEN asset_country = 'UK' AND conversion_country = 'UK' THEN conversions ELSE 0 END) AS UK_Conversions,
    CONCAT(ROUND(SUM(CASE WHEN asset_country = 'UK' AND conversion_country = 'UK' THEN conversions ELSE 0 END) / SUM(CASE WHEN asset_country = 'UK' THEN conversions ELSE 0 END) * 100, 2), '%') AS UK_Conversion_Share,
    SUM(CASE WHEN asset_country = 'UK' AND conversion_country != 'UK' THEN conversions ELSE 0 END) AS Non_UK_Conversions,
    CONCAT(ROUND(SUM(CASE WHEN asset_country = 'UK' AND conversion_country != 'UK' THEN conversions ELSE 0 END) / SUM(CASE WHEN asset_country = 'UK' THEN conversions ELSE 0 END) * 100, 2), '%') AS Non_UK_Conversion_Share,
    SUM(CASE WHEN asset_country = 'DE' AND conversion_country = 'DE' THEN conversions ELSE 0 END) AS DE_Conversions,
    CONCAT(ROUND(SUM(CASE WHEN asset_country = 'DE' AND conversion_country = 'DE' THEN conversions ELSE 0 END) / SUM(CASE WHEN asset_country = 'DE' THEN conversions ELSE 0 END) * 100, 2), '%') AS DE_Conversion_Share,
    SUM(CASE WHEN asset_country = 'DE' AND conversion_country != 'DE' THEN conversions ELSE 0 END) AS Non_DE_Conversions,
    CONCAT(ROUND(SUM(CASE WHEN asset_country = 'DE' AND conversion_country != 'DE' THEN conversions ELSE 0 END) / SUM(CASE WHEN asset_country = 'DE' THEN conversions ELSE 0 END) * 100, 2), '%') AS Non_DE_Conversion_Share
FROM 'campaigns.csv'
GROUP BY attribution_type
ORDER BY UK_Conversions DESC;

,attribution_type,UK_Conversions,UK_Conversion_Share,Non_UK_Conversions,Non_UK_Conversion_Share,DE_Conversions,DE_Conversion_Share,Non_DE_Conversions,Non_DE_Conversion_Share
0,install,104782.0,51.79%,97530.0,48.21%,136158.0,80.56%,32851.0,19.44%
1,retargeting,28385.0,72.42%,10812.0,27.58%,8906.0,90.54%,930.0,9.46%
2,reattr,716.0,88.4%,94.0,11.6%,240.0,89.22%,29.0,10.78%


## Data Analysis: Cross-border traffic for target audience insights
By comparing the with the **_conversion_country_** (where the conversion occurred) and **_country_code_by_phone (user’s phone number)_**, it allows us to detect if there is any target audience issue.

Therefore, in this case, we'll look int 2nd conditions:
- Condition 2: **_country_code_by_phone != conversion_coutry_** > target audience > cross-border traffic
    - Scenario 1: checks for UK users converting outside the UK
        - accesst_country = UK
        - country_code_by_phone = UK
        - conversion_country != UK
    - Scenario 2: checks for non-UK users converting outside the UK, even though the campaign is targeted at the UK
        - accesst_country = UK
        - country_code_by_phone = !UK
        - conversion_country != UK

To understand the increase in Non-UK conversions despite geo-targeting being set for the UK, we can evaluate whether the publishers are targeting the correct audience. We will calculate the cross-border traffic for UK-targeted campaigns by analyzing two key metrics: 1) UK users converting outside the UK, and 2) Non-UK users converting outside the UK. This will help us assess the percentage of cross-border traffic in relation to total conversions for UK campaigns.


### **4. Publisher and Cross-Border Traffic Analysis in UK-Targeted Campaigns (with DE Comparison) **
- Why Do I Want to Know?
    - Understanding target audience misalignment in UK and DE markets helps optimize campaign performance by focusing on the right users, and improving ROI.




### **Findings:**
- UK Market:
    - Many publishers show high UK cross-border conversions, especially PublisherB (44.1%), PublisherS (44.6%), and PublisherG (48.9%), indicating significant misalignment in target audience settings.

- DE Market:
    - PublisherB (20.6%) and PublisherG (20.8%) exhibit notable cross-border conversions, suggesting similar challenges with DE audience targeting.

### **Recommendations:**
- Target Audience Review Needed: Both UK and DE markets face cross-border conversion issues, particularly with key publishers, requiring a thorough review of target audience settings.


In [3]:
-- Look into user insights from non-UK conversions with geo-targeting in the UK
WITH non_uk_conversion_users AS (
    SELECT 
        publisher,
        -- Cross-Border Traffic for UK Users Converting Outside the UK
        SUM(CASE WHEN
			country_code_by_phone = 'UK' AND conversion_country != 'UK' 
            THEN conversions ELSE 0 END) AS UK_Phone_Cross_Border_Conversions,

        -- Cross-Border Traffic for Non-UK Users Converting Outside the UK
        SUM(CASE WHEN
			country_code_by_phone != 'UK' AND conversion_country != 'UK' 
            THEN conversions ELSE 0 END) AS Non_UK_Phone_Cross_Border_Conversions,

        -- Total Conversions for UK Campaigns
        SUM(conversions) AS Total_UK_Conversions
    FROM 'campaigns.csv'
    WHERE asset_country = 'UK'
    GROUP BY publisher
),

non_de_conversion_users AS (
    SELECT 
        publisher,
        -- Cross-Border Traffic for DE Users Converting Outside Germany
        SUM(CASE WHEN
			country_code_by_phone = 'DE' AND conversion_country != 'DE' 
            THEN conversions ELSE 0 END) AS DE_Phone_Cross_Border_Conversions,

        -- Cross-Border Traffic for Non-DE Users Converting Outside Germany
        SUM(CASE WHEN
			country_code_by_phone != 'DE' AND conversion_country != 'DE' 
            THEN conversions ELSE 0 END) AS Non_DE_Phone_Cross_Border_Conversions,

        -- Total Conversions for DE Campaigns
        SUM(conversions) AS Total_DE_Conversions
    FROM 'campaigns.csv'
    WHERE asset_country = 'DE'
    GROUP BY publisher
)

-- Join UK and DE CTEs
SELECT 
    COALESCE(non_uk_conversion_users.publisher, non_de_conversion_users.publisher) AS publisher,

    -- UK Metrics
    non_uk_conversion_users.UK_Phone_Cross_Border_Conversions,
    non_uk_conversion_users.Non_UK_Phone_Cross_Border_Conversions,
    non_uk_conversion_users.Total_UK_Conversions,
    
    -- UK Cross-Border Percentage (rounded and concatenated with %)
    CONCAT(ROUND(non_uk_conversion_users.UK_Phone_Cross_Border_Conversions / non_uk_conversion_users.Total_UK_Conversions * 100, 2), '%') AS UK_Phone_Cross_Border_Percentage,
    CONCAT(ROUND(non_uk_conversion_users.Non_UK_Phone_Cross_Border_Conversions / non_uk_conversion_users.Total_UK_Conversions * 100, 2), '%') AS Non_UK_Phone_Cross_Border_Percentage,

    -- DE Metrics
    non_de_conversion_users.DE_Phone_Cross_Border_Conversions,
    non_de_conversion_users.Non_DE_Phone_Cross_Border_Conversions,
    non_de_conversion_users.Total_DE_Conversions,

    -- DE Cross-Border Percentage (rounded and concatenated with %)
    CONCAT(ROUND(non_de_conversion_users.DE_Phone_Cross_Border_Conversions / non_de_conversion_users.Total_DE_Conversions * 100, 2), '%') AS DE_Phone_Cross_Border_Percentage,
    CONCAT(ROUND(non_de_conversion_users.Non_DE_Phone_Cross_Border_Conversions / non_de_conversion_users.Total_DE_Conversions * 100, 2), '%') AS Non_DE_Phone_Cross_Border_Percentage

FROM non_uk_conversion_users 
LEFT JOIN non_de_conversion_users
ON non_uk_conversion_users.publisher = non_de_conversion_users.publisher
ORDER BY non_uk_conversion_users.publisher;


,publisher,UK_Phone_Cross_Border_Conversions,Non_UK_Phone_Cross_Border_Conversions,Total_UK_Conversions,UK_Phone_Cross_Border_Percentage,Non_UK_Phone_Cross_Border_Percentage,DE_Phone_Cross_Border_Conversions,Non_DE_Phone_Cross_Border_Conversions,Total_DE_Conversions,DE_Phone_Cross_Border_Percentage,Non_DE_Phone_Cross_Border_Percentage
0,PublisherA,7959.0,1820.0,29553.0,26.93%,6.16%,2984.0,847.0,46851.0,6.37%,1.81%
1,PublisherB,24911.0,2618.0,56466.0,44.12%,4.64%,12278.0,1032.0,59742.0,20.55%,1.73%
2,PublisherD,1513.0,5294.0,14122.0,10.71%,37.49%,NaN,NaN,NaN,%,%
3,PublisherE,32.0,3.0,127.0,25.2%,2.36%,16.0,4.0,447.0,3.58%,0.89%
4,PublisherF,1150.0,332.0,5271.0,21.82%,6.3%,27.0,5.0,277.0,9.75%,1.81%
5,PublisherG,5544.0,579.0,11335.0,48.91%,5.11%,1437.0,172.0,6904.0,20.81%,2.49%
6,PublisherH,1.0,1.0,32.0,3.13%,3.13%,4.0,7.0,101.0,3.96%,6.93%
7,PublisherI,1480.0,893.0,10643.0,13.91%,8.39%,271.0,135.0,3337.0,8.12%,4.05%
8,PublisherJ,2564.0,2281.0,7546.0,33.98%,30.23%,NaN,NaN,NaN,%,%
9,PublisherK,8925.0,5047.0,29127.0,30.64%,17.33%,3077.0,2650.0,21963.0,14.01%,12.07%


## Data Analysis: UK conversion behavioral insights

### **5.  Evaluate mobility type and activation time to identify key drivers for potential UK/DE conversion growth.**
- Why Do I Want to Know?
    - It helps optimize strategies for increasing UK conversions and improving campaign effectiveness.

### **Findings**
- **TAXI (<2h activation)** is the top driver for UK conversions, while **Private Hire** sees stronger for Non-UK and DE/Non-DE conversions.

In [24]:
SELECT 
    mobility_type,
    duration_install_to_activation,

    -- UK Conversions (campaign targeted at UK, conversion in UK)
    SUM(CASE WHEN asset_country = 'UK' AND conversion_country = 'UK' THEN conversions ELSE 0 END) AS UK_Conversions,

    -- Non-UK Conversions (campaign targeted at UK, conversion outside UK)
    SUM(CASE WHEN asset_country = 'UK' AND conversion_country != 'UK' THEN conversions ELSE 0 END) AS Non_UK_Conversions,

    -- Germany Conversions (campaign targeted at Germany, conversion in Germany)
    SUM(CASE WHEN asset_country = 'DE' AND conversion_country = 'DE' THEN conversions ELSE 0 END) AS DE_Conversions,

    -- Non-Germany Conversions (campaign targeted at Germany, conversion outside Germany)
    SUM(CASE WHEN asset_country = 'DE' AND conversion_country != 'DE' THEN conversions ELSE 0 END) AS Non_DE_Conversions

FROM 'campaigns.csv'
GROUP BY 1,2
ORDER BY UK_Conversions DESC;


,mobility_type,duration_install_to_activation,UK_Conversions,Non_UK_Conversions,DE_Conversions,Non_DE_Conversions
0,TAXI,<2h,59148.0,6638.0,13847.0,1283.0
1,TAXI,<24h,20801.0,1911.0,4473.0,550.0
2,TAXI,no data,16174.0,1013.0,4861.0,591.0
3,Private Hire,<2h,12887.0,58588.0,65219.0,14286.0
4,Other,<2h,9551.0,1520.0,7278.0,865.0
5,Private Hire,<24h,4219.0,17061.0,16105.0,5527.0
6,Other,<24h,3425.0,476.0,2282.0,327.0
7,Private Hire,no data,3015.0,19802.0,28032.0,9559.0
8,Other,no data,2413.0,245.0,1175.0,316.0
9,TAXI,>24h,1836.0,83.0,331.0,70.0


## **Overall Conclusions**
- UK & Non-UK MoM Conversion Performances: 
    - UK conversions dropped from 83.5% to 28.2%, while non-UK conversions surged to 71.8%, indicating geo-targeting issues and cross-border traffic.

- Publisher Geo-Targeting Issues: 
    - Publishers like PublisherS (54%) and PublisherT (52%) show more geo-targeting mismatches in UK campaigns, while DE campaigns have fewer issues.

- Attribution Type Performance: 
    - The Install type is split nearly 50-50 between UK and non-UK conversions, signaling a need for better geo-targeting, while Retargeting and Reattribution perform better in the UK.

- Target Audience from Cross-Border: 
    - High cross-border conversions from key publishers - PublisherB (44.1%), PublisherS (44.6%), and PublisherG (48.9%) suggest audience targeting needs a review in both UK and DE markets.

- Mobility Type: 
    - TAXI with <2h activation boosts UK conversions, while Private Hire drives non-UK and DE conversions.



## **Limitations**
- Missing Data on Phone Country Codes: 
    - Some users don’t have phone country code data, which makes it harder to accurately analyze cross-border traffic and target audience behavior.


- Limited Data Granularity:
    - The dataset gives a good overview at the publisher and mobility type level, but we’re missing deeper details, like user demographics and campaign specifics, which would help pinpoint why certain campaigns are underperforming.



In [20]:
-- how mismatches look like from publisher
SELECT 
    publisher,
    
    -- UK Campaigns
    SUM(CASE WHEN asset_country = 'UK' AND conversion_country = 'UK' THEN conversions ELSE 0 END) AS UK_Conversions,
    SUM(CASE WHEN asset_country = 'UK' AND conversion_country != 'UK' THEN conversions ELSE 0 END) AS Non_UK_Conversions,
    
    -- Mismatches for UK 
    SUM(CASE WHEN asset_country = 'UK' AND conversion_country != country_code_by_phone THEN conversions ELSE 0 END) AS UK_Mismatched_Conversions,
    CONCAT(ROUND(
        SUM(CASE WHEN asset_country = 'UK' AND conversion_country != country_code_by_phone THEN conversions ELSE 0 END) /
        SUM(CASE WHEN asset_country = 'UK' THEN conversions ELSE 0 END) * 100, 2), '%') AS UK_Mismatched_Conversion_Rate,

    -- DE Campaigns
    SUM(CASE WHEN asset_country = 'DE' AND conversion_country = 'DE' THEN conversions ELSE 0 END) AS DE_Conversions,
    SUM(CASE WHEN asset_country = 'DE' AND conversion_country != 'DE' THEN conversions ELSE 0 END) AS Non_DE_Conversions,

    --Mismatches for DE 
    SUM(CASE WHEN asset_country = 'DE' AND conversion_country != country_code_by_phone THEN conversions ELSE 0 END) AS DE_Mismatched_Conversions,
    CONCAT(ROUND(
        SUM(CASE WHEN asset_country = 'DE' AND conversion_country != country_code_by_phone THEN conversions ELSE 0 END) /
        SUM(CASE WHEN asset_country = 'DE' THEN conversions ELSE 0 END) * 100, 2), '%') AS DE_Mismatched_Conversion_Rate

FROM 'campaigns.csv'
GROUP BY 1
ORDER BY UK_Mismatched_Conversions DESC, DE_Mismatched_Conversions DESC;


,publisher,UK_Conversions,Non_UK_Conversions,UK_Mismatched_Conversions,UK_Mismatched_Conversion_Rate,DE_Conversions,Non_DE_Conversions,DE_Mismatched_Conversions,DE_Mismatched_Conversion_Rate
0,PublisherB,28937.0,27529.0,27782.0,49.2%,46432.0,13310.0,13319.0,22.29%
1,PublisherS,19123.0,23007.0,24652.0,58.51%,15430.0,4317.0,5617.0,28.44%
2,PublisherK,15155.0,13972.0,16759.0,57.54%,16236.0,5727.0,9333.0,42.49%
3,PublisherA,19774.0,9779.0,12951.0,43.82%,43020.0,3831.0,10543.0,22.5%
4,PublisherT,6034.0,6623.0,7576.0,59.86%,7809.0,3071.0,4742.0,43.58%
5,PublisherG,5212.0,6123.0,6401.0,56.47%,5295.0,1609.0,2049.0,29.68%
6,PublisherD,7315.0,6807.0,5888.0,41.69%,0.0,0.0,0.0,%
7,PublisherJ,2701.0,4845.0,5531.0,73.3%,0.0,0.0,0.0,%
8,PublisherO,4849.0,3967.0,4225.0,47.92%,2494.0,834.0,1047.0,31.46%
9,PublisherI,8270.0,2373.0,2998.0,28.17%,2931.0,406.0,771.0,23.1%


In [21]:
-- look into user insights from non-uk conversions with geo-tarketing in UK
WITH non_uk_conversion_users AS (
    SELECT 
        publisher,
        -- Cross-Border Traffic for UK Users Converting Outside the UK
        SUM(CASE WHEN
			country_code_by_phone = 'UK' AND conversion_country != 'UK' 
            THEN conversions ELSE 0 END) AS UK_Phone_Cross_Border_Conversions,

        -- Cross-Border Traffic for Non-UK Users Converting Outside the UK
        SUM(CASE WHEN
			country_code_by_phone != 'UK' AND conversion_country != 'UK' 
            THEN conversions ELSE 0 END) AS Non_UK_Phone_Cross_Border_Conversions,

        -- Total Conversions for Non-UK Campaigns
        SUM(CASE WHEN
			conversion_country != 'UK' 
            THEN conversions ELSE 0 END) AS Total_Non_UK_Conversions
    FROM 'campaigns.csv'
    WHERE asset_country = 'UK'
    GROUP BY publisher
),

non_de_conversion_users AS (
    SELECT 
        publisher,
        -- Cross-Border Traffic for DE Users Converting Outside Germany
        SUM(CASE WHEN
			country_code_by_phone = 'DE' AND conversion_country != 'DE' 
            THEN conversions ELSE 0 END) AS DE_Phone_Cross_Border_Conversions,

        -- Cross-Border Traffic for Non-DE Users Converting Outside Germany
        SUM(CASE WHEN
			country_code_by_phone != 'DE' AND conversion_country != 'DE' 
            THEN conversions ELSE 0 END) AS Non_DE_Phone_Cross_Border_Conversions,

        -- Total Conversions for DE Campaigns
        SUM(CASE WHEN
			conversion_country != 'DE' 
            THEN conversions ELSE 0 END) AS Total_Non_DE_Conversions
    FROM 'campaigns.csv'
    WHERE asset_country = 'DE'
    GROUP BY publisher
)

-- Join UK and DE CTEs
SELECT 
    COALESCE(non_uk_conversion_users.publisher, non_de_conversion_users.publisher) AS publisher,

    -- UK Metrics
    non_uk_conversion_users.UK_Phone_Cross_Border_Conversions,
    non_uk_conversion_users.Non_UK_Phone_Cross_Border_Conversions,
    non_uk_conversion_users.Total_Non_UK_Conversions,
    -- UK Cross-Border Percentage
    (non_uk_conversion_users.UK_Phone_Cross_Border_Conversions / non_uk_conversion_users.Total_Non_UK_Conversions) AS UK_Phone_Cross_Border_Percentage,
    (non_uk_conversion_users.Non_UK_Phone_Cross_Border_Conversions / non_uk_conversion_users.Total_Non_UK_Conversions) AS Non_UK_Phone_Cross_Border_Percentage,

    -- DE Metrics
    non_de_conversion_users.DE_Phone_Cross_Border_Conversions,
    non_de_conversion_users.Non_DE_Phone_Cross_Border_Conversions,
    non_de_conversion_users.Total_Non_DE_Conversions,
    -- DE Cross-Border Percentage
    (non_de_conversion_users.DE_Phone_Cross_Border_Conversions / non_de_conversion_users.Total_Non_DE_Conversions) AS DE_Phone_Cross_Border_Percentage,
    (non_de_conversion_users.Non_DE_Phone_Cross_Border_Conversions / non_de_conversion_users.Total_Non_DE_Conversions) AS Non_DE_Phone_Cross_Border_Percentage

FROM non_uk_conversion_users 
LEFT JOIN non_de_conversion_users
ON non_uk_conversion_users.publisher = non_de_conversion_users.publisher
ORDER BY non_uk_conversion_users.publisher;


,publisher,UK_Phone_Cross_Border_Conversions,Non_UK_Phone_Cross_Border_Conversions,Total_Non_UK_Conversions,UK_Phone_Cross_Border_Percentage,Non_UK_Phone_Cross_Border_Percentage,DE_Phone_Cross_Border_Conversions,Non_DE_Phone_Cross_Border_Conversions,Total_Non_DE_Conversions,DE_Phone_Cross_Border_Percentage,Non_DE_Phone_Cross_Border_Percentage
0,PublisherA,7959.0,1820.0,9779.0,0.813887,0.186113,2984.0,847.0,3831.0,0.778909,0.221091
1,PublisherB,24911.0,2618.0,27529.0,0.904900,0.095100,12278.0,1032.0,13310.0,0.922464,0.077536
2,PublisherD,1513.0,5294.0,6807.0,0.222271,0.777729,NaN,NaN,NaN,NaN,NaN
3,PublisherE,32.0,3.0,35.0,0.914286,0.085714,16.0,4.0,20.0,0.800000,0.200000
4,PublisherF,1150.0,332.0,1482.0,0.775978,0.224022,27.0,5.0,32.0,0.843750,0.156250
5,PublisherG,5544.0,579.0,6123.0,0.905439,0.094561,1437.0,172.0,1609.0,0.893101,0.106899
6,PublisherH,1.0,1.0,2.0,0.500000,0.500000,4.0,7.0,11.0,0.363636,0.636364
7,PublisherI,1480.0,893.0,2373.0,0.623683,0.376317,271.0,135.0,406.0,0.667488,0.332512
8,PublisherJ,2564.0,2281.0,4845.0,0.529205,0.470795,NaN,NaN,NaN,NaN,NaN
9,PublisherK,8925.0,5047.0,13972.0,0.638778,0.361222,3077.0,2650.0,5727.0,0.537280,0.462720


In [22]:
SELECT 
    publisher,

    -- Cross-Border Traffic for UK Users Converting Outside the UK
    SUM(CASE 
        WHEN asset_country = 'UK' AND country_code_by_phone = 'UK' AND conversion_country != 'UK' 
        THEN conversions ELSE 0 END) AS UK_Phone_Cross_Border_Conversions,

    -- Cross-Border Traffic for Non-UK Users Converting Outside the UK
    SUM(CASE 
        WHEN asset_country = 'UK' AND country_code_by_phone != 'UK' AND conversion_country != 'UK' 
        THEN conversions ELSE 0 END) AS Non_UK_Phone_Cross_Border_Conversions,

    -- Total Conversions for UK Campaigns
    SUM(CASE WHEN asset_country = 'UK' THEN conversions ELSE 0 END) AS Total_UK_Campaign_Conversions,

    -- Percentage of Cross-Border Traffic for UK Phone Users
    (SUM(CASE 
        WHEN asset_country = 'UK' AND country_code_by_phone = 'UK' AND conversion_country != 'UK' 
        THEN conversions ELSE 0 END) / SUM(CASE WHEN asset_country = 'UK' THEN conversions ELSE 0 END)) AS UK_Phone_Cross_Border_Rate,

    -- Percentage of Cross-Border Traffic for Non-UK Phone Users
    (SUM(CASE 
        WHEN asset_country = 'UK' AND country_code_by_phone != 'UK' AND conversion_country != 'UK' 
        THEN conversions ELSE 0 END) / SUM(CASE WHEN asset_country = 'UK' THEN conversions ELSE 0 END)) AS Non_UK_Phone_Cross_Border_Rate

FROM 'campaigns.csv'
GROUP BY publisher
ORDER BY UK_Phone_Cross_Border_Conversions DESC;

,publisher,UK_Phone_Cross_Border_Conversions,Non_UK_Phone_Cross_Border_Conversions,Total_UK_Campaign_Conversions,UK_Phone_Cross_Border_Rate,Non_UK_Phone_Cross_Border_Rate
0,PublisherB,24911.0,2618.0,56466.0,0.441168,0.046364
1,PublisherS,18796.0,4211.0,42130.0,0.446143,0.099953
2,PublisherK,8925.0,5047.0,29127.0,0.306417,0.173276
3,PublisherA,7959.0,1820.0,29553.0,0.269313,0.061584
4,PublisherG,5544.0,579.0,11335.0,0.489105,0.051081
5,PublisherO,3509.0,458.0,8816.0,0.398026,0.051951
6,PublisherT,3383.0,3240.0,12657.0,0.267283,0.255985
7,PublisherJ,2564.0,2281.0,7546.0,0.339783,0.302279
8,PublisherD,1513.0,5294.0,14122.0,0.107138,0.374876
9,PublisherI,1480.0,893.0,10643.0,0.139059,0.083905


In [23]:
SELECT 
    mobility_type,
    duration_install_to_activation,

    -- Cross-Border Traffic for UK Users Converting Outside the UK
    SUM(CASE 
        WHEN asset_country = 'UK' AND country_code_by_phone = 'UK' AND conversion_country != 'UK' 
        THEN conversions ELSE 0 END) AS UK_Phone_Cross_Border_Conversions,

    -- Cross-Border Traffic for Non-UK Users Converting Outside the UK
    SUM(CASE 
        WHEN asset_country = 'UK' AND country_code_by_phone != 'UK' AND conversion_country != 'UK' 
        THEN conversions ELSE 0 END) AS Non_UK_Phone_Cross_Border_Conversions,

    -- Total Conversions for UK Campaigns
    SUM(CASE WHEN asset_country = 'UK' THEN conversions ELSE 0 END) AS Total_UK_Campaign_Conversions,

    -- Percentage of Cross-Border Traffic for UK Phone Users
    (SUM(CASE 
        WHEN asset_country = 'UK' AND country_code_by_phone = 'UK' AND conversion_country != 'UK' 
        THEN conversions ELSE 0 END) / SUM(CASE WHEN asset_country = 'UK' THEN conversions ELSE 0 END)) * 100 AS UK_Phone_Cross_Border_Rate,

    -- Percentage of Cross-Border Traffic for Non-UK Phone Users
    (SUM(CASE 
        WHEN asset_country = 'UK' AND country_code_by_phone != 'UK' AND conversion_country != 'UK' 
        THEN conversions ELSE 0 END) / SUM(CASE WHEN asset_country = 'UK' THEN conversions ELSE 0 END)) * 100 AS Non_UK_Phone_Cross_Border_Rate

FROM 'campaigns.csv'
GROUP BY mobility_type, duration_install_to_activation
ORDER BY UK_Phone_Cross_Border_Conversions DESC;



,mobility_type,duration_install_to_activation,UK_Phone_Cross_Border_Conversions,Non_UK_Phone_Cross_Border_Conversions,Total_UK_Campaign_Conversions,UK_Phone_Cross_Border_Rate,Non_UK_Phone_Cross_Border_Rate
0,Private Hire,<2h,42158.0,16430.0,71475.0,58.982861,22.987058
1,Private Hire,no data,18023.0,1779.0,22817.0,78.989350,7.796818
2,Private Hire,<24h,11505.0,5556.0,21280.0,54.064850,26.109023
3,TAXI,<2h,5003.0,1635.0,65786.0,7.604962,2.485331
4,TAXI,<24h,1224.0,687.0,22712.0,5.389222,3.024833
5,Other,<2h,982.0,538.0,11071.0,8.870021,4.859543
6,TAXI,no data,920.0,93.0,17187.0,5.352883,0.541107
7,Private Hire,>24h,824.0,229.0,1282.0,64.274571,17.862715
8,Other,<24h,272.0,204.0,3901.0,6.972571,5.229428
9,Other,no data,209.0,36.0,2658.0,7.863055,1.354402
